# Homework 3

Daniil Sherki

# Problem statement

A hydrocarbon mixture with the following overall composition is flashed in a
separator at 50 psia and 100°F. (You need to use Cox Chart)

| Component  | $z_i$ |
|------------|-------|
| $C_3$      | 0.20  |
| $i-C_4$    | 0.10  |
| $n-C_4$    | 0.10  |
| $i-C_5$    | 0.20  |
| $n-C_5$    | 0.20  |
| $C_6$      | 0.20  |

Assuming an ideal solution behavior, perform flash calculations.

# Solution

## Import libraries and initial data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# conversion functions to convert one measurement systems into other

class Conversion():
    def __init__(self):
        pass

    def R2F(self, x):
        '''
        ** Temperature **
        Rankine degrees into Fahrenheit degrees
        :param x: float; // R degress
        :return: float. // F degrees
        '''
        return x - 459.67

    def F2R(self, x):
        '''
        ** Temperature **
        Fahrenheit degrees into Rankine degrees
        :param x: float; // F degress
        :return: float. // R degrees
        '''
        return x + 459.67

    def psia2bar(self, x):
        '''
        ** Pressure **
        pisa into bar
        1 psi = 0.0689475729 bar
        :param x: float; // psia
        :return: float. // bar
        '''
        return 0.0689475729 * x

    def bar2psia(self, x):
        '''
        ** Pressure **
        bar into psia
        1 bar = 14.503773773 psi
        :param x: float; // bar
        :return: float. // psia
        '''
        return 14.503773773 * x

c = Conversion()

In [4]:
df = pd.DataFrame({'Component':['C3', 'iC4', 'nC4', 'iC5', 'nC5', 'C6'],
                   'z_i': [0.2, 0.1, 0.1, 0.2, 0.2, 0.2]})
df

,Component,z_i
0,C3,0.2
1,iC4,0.1
2,nC4,0.1
3,iC5,0.2
4,nC5,0.2
5,C6,0.2


# 1. Determining the vapor pressure $p_{vi}$ from the Cox chart and calculating the equilibrium ratios.

$$ K_i = \frac{y_i}{x_i} $$

where
$K_i$ - equilibrium ratio of component $i$
$y_i$ - mole fraction of component $i$ in the gas phase
$x_i$ - mole fraction of component $i$ in the liquid phase

By definition,

$$ p_i = x_i p_{vi} $$

$$ p_i = y_i p $$

$p_i$ - partial pressure of a component i, psia
$p_{vi}$ - vapor pressure of component i, psia

Thus,

$$ K_i = \frac{y_i}{x_i} = \frac{p_{vi}}{p}$$

In [9]:
p = 50
df['p_vi_100F'] = [190, 72.2, 51.6, 20.44, 15.57, 4.956]
df['K_i'] = df['p_vi_100F'] / p
df

,Component,z_i,p_vi_100F,K_i
0,C3,0.2,190.000,3.80000
1,iC4,0.1,72.200,1.44400
2,nC4,0.1,51.600,1.03200
3,iC5,0.2,20.440,0.40880
4,nC5,0.2,15.570,0.31140
5,C6,0.2,4.956,0.09912


# 2. Solving equations for $n_v$ using the Newton-Raphson method.

By definition,

$$ n_L + n_v = n_t$$

$n_t$ - total number of moles of the hydrocarbon mixture, lb-mol
$n_L$ - total number of moles in the liquid phase
$n_v$ - total number of moles in the vapor (gas) phase

A material balance on the ith component results in

$$z_i n_t = x_i n_L + y_i n_v$$

$z_i n_t$ - total number of moles of component $i$ in the system
$x_i n_L$ - total number of moles of component $i$ in the liquid phase
$y_i n_v$ - total number of moles of component $i$ in the vapor phase

Also, by the definition of the total mole fraction in a hydrocarbon system, we may write:

$$ \sum_i z_i = 1$$
$$ \sum_i x_i = 1$$
$$ \sum_i y_i = 1$$

Thus,
$$ n_L + n_v = 1$$

And
$$x_i n_L + y_i n_v = z_i$$
$$x_i n_L + (x_i K_i) n_v = z_i$$

$$x_i = \frac{z_i}{n_L + n_v K_i}$$
$$y_i = \frac{z_i K_i}{n_L + n_v K_i} = x_i K_i$$

$$\sum_i x_i = \sum_i \frac{z_i}{n_L + n_v K_i}$$
$$\sum_i y_i = \sum_i \frac{z_i K_i}{n_L + n_v K_i} = x_i K_i$$
$$\sum_i y_i - \sum_i x_i = 0$$

$$\sum_i \frac{z_i K_i}{n_L + n_v K_i } - \sum_i \frac{z_i}{n_L + n_v K_i} = 0$$

$$\sum_i \frac{z_i (K_i - 1)}{n_L + n_v K_i } = 0$$

$$n_L = 1 - n_v$$

And we need solve this equation using Newton-Raphson method
$$f(n_v) = \sum_i \frac{z_i (K_i - 1)}{n_v(K_i - 1) + 1} = 0$$

$$f'(n_v) = \sum_i \frac{z_i (K_i - 1)^2}{\left[ n_v(K_i - 1) + 1 \right]^2}$$

In [16]:
def f_n_v(n_v, z=df['z_i'], K=df['K_i']):
    '''f(n_v)
    :param n_v: float;
    :param z: pandas.Series;
    :param K: pandas.Series;
    :return: flaot.
    '''
    out = 0
    for i in range(len(z)):
       out += z[i]*(K[i]-1)/(n_v * (K[i] - 1)+1)
    return out

def grad_f_n_v(n_v, z=df['z_i'], K=df['K_i']):
   '''
   f'(n_v)
   :param n_v: float;
   :param z: pandas.Series;
   :param K: pandas.Series;
   :return: flaot.
   '''
   out = 0
   for i in range(len(z)):
       out += - z[i]*(K[i]-1)**2/(n_v * (K[i] - 1)+1)**2
   return out

def Newton_Raphson_method(x0 = 0.5, func = f_n_v, grad_func = grad_f_n_v, disp = True):
        '''
        Newton-Raphson method to find zero value of reduced density equation
        f(x) = 0
        :param x0: float; // initial point
        :param func: function; // function, which we want to make equal zero
        :param grad_func: function; // gradient function
        :param disp: Boolean; // do we need disp iter res or not
        :return: (float, float). // value function and x*
        '''
        i = 0
        x = x0
        x_prev = 0
        while abs(x - x_prev) > 1e-12:
            if disp:
                print(f'iter:{i}, n_v:{x}, f(n_v):{func(x)}')
            i+=1
            x_prev = x
            x = x_prev - func(x)/grad_func(x)
        if disp:
            print(f'iter:{i}, n_v:{x}, f(n_v):{func(x)}')
        return func(x), x

f_n_v_val, n_v = Newton_Raphson_method()

iter:0, n_v:0.5, f(n_v):-0.4329324407046753
iter:1, n_v:0.13447869641957455, f(n_v):-0.033338684842064786
iter:2, n_v:0.1078713170015532, f(n_v):0.0010190875883279715
iter:3, n_v:0.10863610135783802, f(n_v):9.719274248476406e-07
iter:4, n_v:0.1086368321428544, f(n_v):8.838207943284715e-13
iter:5, n_v:0.10863683214351894, f(n_v):-5.551115123125783e-17


In [17]:
print(f'This, n_v = {n_v}')

This, n_v = 0.10863683214351894


# 3. Solution for $n_L$

In [18]:
n_L = 1- n_v
print(f'This, n_L = {n_L}')

This, n_L = 0.8913631678564811


# 4. Solution for $x_i$ and $y_i$ to yield


$$x_i = \frac{z_i}{n_L + n_v K_i}$$
$$y_i = x_i K_i$$

In [19]:
df['x_i'] = df['z_i']/(n_L + n_v * df['K_i'])
df['y_i'] = df['x_i'] * df['K_i']
df

,Component,z_i,p_vi_100F,K_i,x_i,y_i
0,C3,0.2,190.000,3.80000,0.153353,0.582740
1,iC4,0.1,72.200,1.44400,0.095398,0.137755
2,nC4,0.1,51.600,1.03200,0.099654,0.102842
3,iC5,0.2,20.440,0.40880,0.213727,0.087372
4,nC5,0.2,15.570,0.31140,0.216171,0.067316
5,C6,0.2,4.956,0.09912,0.221697,0.021975
